In [ ]:
%matplotlib inline

import warnings

import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from scipy.constants import h, c, e, k
from scipy.integrate import simps
from scipy.interpolate import interp1d
from scipy.optimize import differential_evolution

q = e

warnings.filterwarnings("ignore")

In [ ]:
def wl_to_E(wl):
    """Convert wavelength to energy in eV.
    
    Parameters
    ----------
    wl : float or array
        wavelength in nm
    
    Returns
    -------
    E : float or array
        energy in eV
    """
    return sp.constants.h * sp.constants.c / (q * wl * 1e-9)


def E_to_wl(E):
    """Convert energy to wavelength.
    
    Parameters
    ----------
    E : float or array
        energy in eV
    
    Returns
    -------
    wl : float or array
        wavelength in nm
    """
    return sp.constants.h * sp.constants.c / (q * E * 1e-9)


def wn_to_wl(wn):
    """Convert wavenumber to wavelength.
    
    Parameters
    ----------
    wn : float or array
        wavenumber in /cm
    
    Returns
    -------
    wl : float or array
        wavelegnth in nm
    """
    return 1 / (wn * 100 * 1e-9)


def wn_to_E(wn):
    """Convert wavenumber to energy.
    
    Parameters
    ----------
    wn : float or array
        wavenumber in /cm
    
    Returns
    -------
    E : float or array
        energy in eV
    """
    return wl_to_E(wn_to_wl(wn))


def R_corrected(Rsample, R0, R100, Rref=1):
    """Calculate R from measured data.
    
    Parameters
    ----------
    Rsample : array
        measured sample reflection
    R0 : array
        measured reflection without sample
    R100: array
        measured reflection of reference mirror
    Rref : array
        calibrated reflectance of reference mirror
    
    Returns
    -------
    R : array
        reflectance
    """
    return (Rsample - R0) * Rref / (R100 - R0)
    
    
def T_corrected(Tsample, T0, T100):
    """Calculate T from measured data.
    
    Parameters
    ----------
    Tsample : array
        measured sample transmission
    T0 : array
        measured transmission without sample
    TR100: array
        measured transmission of reference mirror
    
    Returns
    -------
    T : array
        transmittance
    """
    return (Tsample - T0) / (T100 - T0)
    
    
def OD(Tsample, Rsample, T0, T100, R0, R100, Rref=1):
    """Calculate optical density from measured T and R data.
    
    Parameters
    ----------
    Tsample : array
        measured sample transmission
    Rsample : array
        measured sample reflection
    T0 : array
        measured background (blocked beam) transmission 
    T100 : array
        measured transmission without sample
    R0 : array
        measured reflection without sample
    R100: array
        measured reflection of reference mirror
    Rref : array
        calibrated reflectance of reference mirror
    
    Returns
    -------
    OD : array
        optical density
    """
    # calculate transmittance
    T = T_corrected(Tsample, T0, T100)
    
    # calculate reflectance
    R = R_corrected(Rsample, R0, R100, Rref)
    
    return -np.log(T / (1 - R))


def alpha(OD, d):
    """Calculate absorption coefficient in /cm.
    
    Parameters
    ----------
    OD : float or array
        optical density
    d : float
        film thickness in cm
    
    Returns
    -------
    alpha : float or array
        absorption coefficient in /cm"""
    
    return OD / d


def a_Br(E, Eg, Eb, b, a0, G, dE, nmax, th, c=1):
    """
    Calculate the absorption coefficient at the bandedge of a 
    semiconductor based on Elliot's model.
    
    Based on Saba et al.,
    https://www.nature.com/articles/ncomms6049.
    
    Parameters
    ----------
    E : float or array
        photon energy in eV
    Eg : float
        bandgap energy in eV
    Eb : float
        exciton binding energy in eV
    b : float
        band non-parabolicity factor
    a0 : float
        absorption coefficient prefactor in cm^-1
    G : float
        broadening parameter in eV
    dE : float
        photon energy spacing between data points in eV
    nmax : int
        maximum number of excitonic peaks to include
    th : float
        lower threshold for bounding normalised convolution window
    c : float (optional)
        density of states factor. This cancels out in the calculation
        but is formally present in the derivation.
    
    Returns
    -------
    a : array
        absorption coefficient in cm^-1
    """
    # calculate maximum and minimum energies to bound the convoluted function
    Emax = G * np.log((1 / th) + np.sqrt((1 / th)**2 - 1)) + Eg
    Emin = 2 * Eg - Emax
       
    # calculate number of points
    n_kern = int((Emax - Emin) / dE) + 1

    # calculate kernel
    E_kern = np.linspace(Emin, Emax, n_kern)
    y_kern = 1 / np.cosh((E_kern - Eg) / G)

    # kernel width
    w_kern = np.shape(y_kern)[0]
    
    # pad independent variable for original function
    Epad = np.pad(E, int(w_kern / 2), 'linear_ramp',
                  end_values=(np.min(E) - int(w_kern / 2) * dE,
                              np.max(E) + int(w_kern / 2) * dE))

    # density of states
    g = np.piecewise(Epad, [Epad <= Eg, Epad > Eg], [0, lambda Epad: c * np.sqrt(Epad - Eg) / (1 - b * (Epad - Eg))])
    
    # free-carrier absorption coefficient without enhancement
    a_free = a0 * g / (Epad * c)
    
    # energy scaled by exciton binding energy
    D = np.piecewise(Epad, [Epad <= Eg, Epad > Eg], [0, lambda Epad: np.sqrt(Eb / (Epad - Eg))])
    
    # sommerfield enhancement factor
    S = np.piecewise(D, [D <= 0, D > 0], [0, lambda D: 2 * np.pi * D / (1 - np.exp(-2 * np.pi * D))])
    
    # continuum absorption coefficient with enhancement
    a_c = a_free * S
    
    # exciton absorption coefficient
    a_x = np.zeros(len(Epad))
    for n in range(1, nmax + 1):
        Enx = Eg - Eb / n**2
        dirac_delta = np.where(Epad==Enx, 1, 0)
        a_x += a0 * 4 * np.pi * Eb**(3/2) * dirac_delta / (Enx * n**3)
    
    # total absorption coefficient
    a = a_c + a_x

    # free-carrier absorption coefficient convoluted with a 
    # broadening window
    a_free_Br = np.convolve(y_kern, a_free, 'valid') / sum(y_kern)

    # the number of points in output may be different to the original
    # data depending on whether the number of points in the kernel is
    # odd or even
    if w_kern % 2 == 0:
        a_free_Br = a_free_Br[:-1]
    
    # continuum absorption coefficient by convoluted with a 
    # broadening window
    a_c_Br = np.convolve(y_kern, a_c, 'valid') / sum(y_kern)

    # the number of points in output may be different to the original
    # data depending on whether the number of points in the kernel is
    # odd or even
    if w_kern % 2 == 0:
        a_c_Br = a_c_Br[:-1]
    
    # exciton absorption coefficient convoluted with broadening window 
    a_x_Br = np.zeros(len(E))
    for n in range(1, nmax + 1):
        Enx = Eg - Eb / n**2
        br = 1 / np.cosh((E - Enx) / G)
        N = simps(br, E)

        a_x_Br += a0 * 4 * np.pi * Eb**(3/2) * br / (E * n**3 * N)

    return a_free_Br, a_c_Br, a_x_Br


def a_Br_min(x, *p):
    """
    Least-squares minimisation function for fitting absorption
    coefficient data.
    
    Parameters
    ----------
    x : list
        vector of parameters to fit
    *p : list
        additional arguments required to fully define the function
    
    Returns
    -------
    ssr : float
        sum of squared residuals
    """
    # unpack fitting parameters and additional arguments
    Eg, Eb, b, a0, G = x
    dE, nmax, th, E, ydata = p
    
    # calculated absorption coefficient according to Elliot's model
    _, a_c_Br, a_x_Br = a_Br(E, Eg, Eb, b, a0, G, dE, nmax, th)
    
    # calculate residual
    res = a_c_Br + a_x_Br - ydata
    
    return np.sum(res**2)

In [ ]:
# film thickness in cm
thickness = 300e-7

# measurement file names
# if files are in a different folder to this script, these should be the full paths to the files
file_R100 = "R100.tsv"       # 100% reflection
file_R0 = "R0.tsv"           # 0% reflection
file_T100 = "T100.tsv"       # 100% transmission
file_T0 = "T0.tsv"           # 0% transmission
file_Rsample = "Rsample.tsv" # Sample reflection
file_Tsample = "Tsample.tsv" # Sample transmission

# load measurement data files
# assumes wavelengths in first column, measurement in second column, column names as first row, tab-delimited
R100 = np.genfromtxt(file_R100, skip_header=1, delimiter='\t')
R0 = np.genfromtxt(file_R0, skip_header=1, delimiter='\t')
T100 = np.genfromtxt(file_T100, skip_header=1, delimiter='\t')
T0 = np.genfromtxt(file_T0, skip_header=1, delimiter='\t')
Rsample = np.genfromtxt(file_Rsample, skip_header=1, delimiter='\t')
Tsample = np.genfromtxt(file_Tsample, skip_header=1, delimiter='\t')

# reflectance reference file name (calibration data for reflectance standard)
file_Rref = "Rref.tsv"

# load and interpolate reflectance reference
# assumes wavelengths in first column, R in second column, column names as first row, tab-delimited
Rref = np.genfromtxt(file_Rref, skip_header=1, delimiter='\t')
f_Rref = sp.interpolate.interp1d(Rref[:,0], Rref[:,1], kind='cubic', bounds_error=False, fill_value=0)
wls = R100[:,0] # get interpolation wavelengths as first columns of any measurement file
Rref_int = f_Rref(wls)

# calculate optical density
od = OD(Tsample[:,1], Rsample[:,1], T0[:,1], T100[:,1], R0[:,1], R100[:,1], Rref_int)

# calculate absorption coefficient
a = alpha(od, thickness)

In [ ]:
# interpolate absorption coefficient evenly in energy
mask = np.where((abs_Es >= 1.1) & (abs_Es < 2.2))
f_a0 = sp.interpolate.interp1d(abs_Es[mask], a0[mask], kind='cubic', bounds_error=False, fill_value=0)
abs_Es_new, dE = np.linspace(1.1, 2.0, 101, endpoint=True, retstep=True)
a0_int = f_a0(abs_Es_new)

th = 1e-3
nmax=6

mask = np.where((abs_Es_new >= 1.2) & (abs_Es_new < 1.6))
res0 = sp.optimize.differential_evolution(a_Br_min, seed=10,
                                          args=(dE, nmax, th, abs_Es_new[mask], a0_int[mask] - a0_int[mask][0]),
                                          bounds=((1.2, 1.6), (0, 0.2), (0, 2), (0, 1e6), (0, 1)))

print(f'u=0\n---\nEg={res0.x[0]:.2f} eV, '
      f'Eb={res0.x[1]:.2e} eV, '
      f'b={res0.x[2]:.2f}, '
      f'a0={res0.x[3]:.2e} /cm, ' 
      f'G={res0.x[4]:.2e} eV\n'
)

In [ ]:
a0_free_Br_fit, a0_c_Br_fit, a0_x_Br_fit = a_Br(abs_Es_new[mask],
                                                res0.x[0],
                                                res0.x[1],
                                                res0.x[2],
                                                res0.x[3],
                                                res0.x[4],
                                                dE,
                                                nmax,
                                                th)

a0_fit = a0_c_Br_fit + a0_x_Br_fit

fig, ax = plt.subplots()

ax.plot(abs_Es_new, a0_int, color='black', marker='o', markersize=4, markevery=2, lw=1, label='experiment')
ax.plot(abs_Es_new[mask], a0_fit + a0_int[mask][0], color='red', lw=2, label='fit')
ax.set_xlim(1.1, 1.9)
ax.set_ylim(0, 3.9e4)
ax.tick_params(direction='in', top=True, right=True, labelsize='large', color='black', labelcolor='black')
ax.set_ylabel(r'$\alpha\ \mathsf{(cm^{-1})}$', fontsize='large')
ax.set_xlabel('Photon energy (eV)', fontsize='large')
ax.legend(loc=(0.03, 0.63), frameon=False, fontsize='large')
ax.text(1.145, 35500, 'u=0', fontsize='large')

In [ ]:
fig.savefig('elliot_fit.svg', bbox_inches='tight', pad_inches=0.0)